In [2]:
from pathlib import Path
from uuid import uuid4
import json
import os
import sys
from dotenv import load_dotenv
from tqdm import tqdm
import qdrant_client
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer

/Users/mason/opt/anaconda3/envs/psai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables - with fallback and error handling
try:
    load_dotenv("/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/.env")
    # Alternative: use relative path or allow custom path input
    # load_dotenv(Path.cwd().parent / "code" / ".env")
    
    QDRANT_URL = os.getenv("QDRANT_URL")
    QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
    
    if not QDRANT_URL or not QDRANT_API_KEY:
        raise ValueError("QDRANT_URL or QDRANT_API_KEY environment variables not found")
    
    # Initialize Qdrant client and embedding model
    client = qdrant_client.QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
    print(f"Successfully connected to Qdrant at {QDRANT_URL}")
    
    model = SentenceTransformer("all-MiniLM-L6-v2")
    print(f"Loaded embedding model: all-MiniLM-L6-v2 (output dimension: {model.get_sentence_embedding_dimension()})")
except Exception as e:
    print(f"Error during initialization: {e}")
    sys.exit(1)

Successfully connected to Qdrant at https://3031677a-6463-44f9-ba66-42977581720e.us-east-1-0.aws.cloud.qdrant.io
Loaded embedding model: all-MiniLM-L6-v2 (output dimension: 384)


In [4]:
def ensure_collection(name):
    """Create collection if it doesn't exist."""
    try:
        collections = [c.name for c in client.get_collections().collections]
        if name not in collections:
            client.recreate_collection(
                collection_name=name,
                vectors_config=VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE)
            )
            print(f"Created collection: {name}")
        else:
            print(f"Collection {name} already exists")
    except Exception as e:
        print(f"Error ensuring collection {name}: {e}")
        raise

def embed_and_upload(chunks, collection_name):
    """Embed text chunks and upload them to Qdrant."""
    if not chunks:
        print(f"Warning: No chunks to upload to {collection_name}")
        return
    
    print(f"Processing {len(chunks)} chunks for collection {collection_name}...")
    ensure_collection(collection_name)
    
    # Process in batches to avoid memory issues with large datasets
    batch_size = 100
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        texts = [chunk["text"] for chunk in batch]
        
        # Embed texts
        try:
            embeddings = model.encode(texts, show_progress_bar=True).tolist()
        except Exception as e:
            print(f"Error embedding batch starting at index {i}: {e}")
            continue
            
        points = []
        for chunk, embedding in zip(batch, embeddings):
            payload = chunk.get("metadata", {}).copy()
            payload["text"] = chunk["text"]
            points.append(
                PointStruct(
                    id=str(uuid4()),
                    vector=embedding,
                    payload=payload
                )
            )
        
        try:
            client.upload_points(collection_name=collection_name, points=points)
            print(f"Uploaded batch {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1} to {collection_name}")
        except Exception as e:
            print(f"Error uploading batch to {collection_name}: {e}")

def load_book_chunks(book_dir):
    """Load chunks from book JSON files."""
    chunks = []
    book_dir = Path(book_dir)
    if not book_dir.exists():
        print(f"Warning: Book directory {book_dir} does not exist")
        return chunks
        
    json_files = list(book_dir.glob("*.json"))
    print(f"Found {len(json_files)} book JSON files")
    
    for file in tqdm(json_files, desc="Loading book chunks"):
        try:
            with open(file, "r", encoding="utf-8") as f:
                book_data = json.load(f)
                for entry in book_data:
                    chunks.append({
                        "text": entry["text"],
                        "metadata": {
                            "author": entry.get("author", "Unknown"),
                            "book_title": entry.get("book_title", "Unknown"),
                            "publication_year": entry.get("publication_year", "Unknown"),
                            "doc_type": "Phyllis Schlafly Book",
                            "source_file": file.name
                        }
                    })
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    print(f"Loaded {len(chunks)} total book chunks")
    return chunks

def load_psr_chunks(path):
    """Load PSR chunks from JSON file."""
    path = Path(path)
    if not path.exists():
        print(f"Warning: PSR file {path} does not exist")
        return []
        
    try:
        with open(path, "r", encoding="utf-8") as f:
            chunks = json.load(f)
            print(f"Loaded {len(chunks)} PSR chunks from {path}")
            return chunks
    except Exception as e:
        print(f"Error loading PSR chunks from {path}: {e}")
        return []

def load_psc_chunks(psc_dir):
    """Load PSC chunks from JSON files."""
    chunks = []
    psc_dir = Path(psc_dir)
    if not psc_dir.exists():
        print(f"Warning: PSC directory {psc_dir} does not exist")
        return chunks
        
    json_files = list(psc_dir.glob("psc_*.json"))
    print(f"Found {len(json_files)} PSC JSON files")
    
    for file in tqdm(json_files, desc="Loading PSC chunks"):
        try:
            with open(file, "r", encoding="utf-8") as f:
                year_data = json.load(f)
                if "chunks" in year_data:
                    # Add source file metadata
                    for chunk in year_data["chunks"]:
                        if "metadata" not in chunk:
                            chunk["metadata"] = {}
                        chunk["metadata"]["source_file"] = file.name
                    
                    chunks.extend(year_data["chunks"])
                else:
                    print(f"Warning: No 'chunks' key in {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    print(f"Loaded {len(chunks)} total PSC chunks")
    return chunks

# Function to check if paths exist
def check_paths(paths):
    """Check if all paths exist and return valid ones."""
    valid_paths = {}
    for name, path in paths.items():
        path_obj = Path(path)
        if path_obj.exists():
            valid_paths[name] = path_obj
            print(f"✓ {name} path exists: {path}")
        else:
            print(f"✗ {name} path does not exist: {path}")
    return valid_paths

# Main execution
def main():
    # Define paths
    paths = {
        "book_path": "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/books",
        "psr_path": "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/psr_chunks.json",
        "psc_path": "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/psc_chunks"
    }
    
    # Check paths before proceeding
    valid_paths = check_paths(paths)
    
    # Load chunks from valid paths
    all_chunks = {}
    
    if "book_path" in valid_paths:
        book_chunks = load_book_chunks(valid_paths["book_path"])
        all_chunks["book_chunks"] = book_chunks
    
    if "psr_path" in valid_paths:
        psr_chunks = load_psr_chunks(valid_paths["psr_path"])
        all_chunks["psr_chunks"] = psr_chunks
    
    if "psc_path" in valid_paths:
        psc_chunks = load_psc_chunks(valid_paths["psc_path"])
        all_chunks["psc_chunks"] = psc_chunks
    
    # Upload all chunks to Qdrant
    for collection_name, chunks in all_chunks.items():
        print(f"\nProcessing collection: {collection_name}")
        embed_and_upload(chunks, collection_name)
        print(f"Completed upload to {collection_name}")

# # In a Jupyter notebook, you might want to run this directly rather than as a function
# # For production code, using the function is better
# if __name__ == "__main__":
#     main()
# else:
#     # When imported as a module or run in Jupyter
#     print("Script loaded and ready. Run main() to process and upload all chunks.")

In [5]:
main()

✓ book_path path exists: /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/books
✓ psr_path path exists: /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/psr_chunks.json
✓ psc_path path exists: /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/psc_chunks
Found 3 book JSON files


Loading book chunks: 100%|██████████| 3/3 [00:00<00:00, 171.60it/s]

Loaded 693 total book chunks


Loaded 4621 PSR chunks from /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/psr_chunks.json
Found 42 PSC JSON files


Loading PSC chunks: 100%|██████████| 42/42 [00:00<00:00, 119.66it/s]
/var/folders/fc/s2pyk63x5958wcxdtr6xlpwc0000gn/T/ipykernel_27501/391725248.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


...

Uploaded batch 239/241 to psc_chunks


Batches: 100%|██████████| 4/4 [00:02<00:00,  1.86it/s]


Uploaded batch 240/241 to psc_chunks


Batches: 100%|██████████| 2/2 [00:01<00:00,  1.74it/s]


Uploaded batch 241/241 to psc_chunks
Completed upload to psc_chunks


In [6]:
# Check if processing completed successfully
for collection_name in ["book_chunks", "psr_chunks", "psc_chunks"]:
    try:
        info = client.get_collection(collection_name=collection_name)
        print(f"Collection '{collection_name}' contains {info.points_count} points")
    except Exception as e:
        print(f"Error checking collection '{collection_name}': {e}")

Collection 'book_chunks' contains 693 points
Collection 'psr_chunks' contains 4621 points
Collection 'psc_chunks' contains 24034 points
